In [1]:
#First we import the relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#skal se om denne ændring kommer med




SyntaxError: invalid syntax (<ipython-input-1-84295e4e90b8>, line 7)

In [45]:
#Next we define the relevant parameters
SU=0.8 # Det ændrer hvad man vælger
beta=1.0 #Discount factor
B=0.5 #Coefficient til humankapital
T=3 #Antal tidsperioder
max_HK=3 #Maksimalt niveau af uddannelse.
num_points=100

In [43]:
#Next we define the solution of the model:

def sol_bel(SU,B,beta):
    ev0 = np.nan+np.zeros([max_HK,T]) #Generer expected value function.
    pk = np.nan+np.zeros([max_HK,T]) # Choice prob for choosing work
    V_school=np.nan+np.zeros([max_HK,T]) # Choice specific value functions
    V_work=np.nan+np.zeros([max_HK,T]) # Choice specific value functions


#Now I generate the expected value for THE LAST PERIOD from both alternatives and the value function (which is the max of the two)
    V_school[:,T-1]=SU # Last period value from schooling
    for h in range(max_HK):
        np.random.seed(2022)
        x = np.random.normal(size=num_points) 
        V_work[h,T-1]=np.mean(B*h+x) # Last period value from work depending on HK
    
    ev0[:,T-1]=np.maximum(V_school[:,T-1],V_work[:,T-1]) 

#Now I generate the choice probabilities in the last period 
    pk[:,T-1]=ev0[:,T-1]==V_work[:,T-1] #Uden stokastik
#pk[:,T-1]=np.exp(V_work[:,T-1])/(np.exp(V_school[:,T-1])+np.exp(V_work[:,T-1])) # MED EV1 error terms er det logit form expression. 


#Her løser jeg modellen via backwards induction for periode T-1 indtil for periode 0
    for t in range(T-2, -1, -1): #Solve backwards from period T-1(2??), until period 0, backwards
        for h in range(max_HK):
            np.random.seed(2022)
            x = np.random.normal(size=num_points)
            V_school[h,t]=SU + beta*ev0[np.minimum(h+1,max_HK-1),t+1] #Denne viser værdien af skole som er, værdien af SU samt den forventede value function hvor vi opdaterer hk., i t+1
            V_work[h,t]= np.mean(B*h + x) + beta*ev0[h,t+1] #Denne viser værdien af work som er lig state space matricen evalueret der hvor vi er + den forventede value function i periode t+1
            ev0[h,t]=np.maximum(V_work[h,t],V_school[h,t]) #Denne viser expected value function som er lig den maksimale værdi vi får fra den chocie specific value fct.
            pk[h,t]=ev0[h,t]==V_work[h,t] #Uden stokastik
            
    return pk, V_work, V_school, ev0


    

In [44]:
ccp,Val_W,Val_S,EV=sol_bel(SU,B,beta)
print(EV)
print(Val_W)

[[0.8        0.8        0.8       ]
 [0.8        0.8        0.8       ]
 [0.99185454 0.99185454 0.99185454]]
[[-0.00814546 -0.00814546 -0.00814546]
 [ 0.49185454  0.49185454  0.49185454]
 [ 0.99185454  0.99185454  0.99185454]]


In [51]:
#Nu skal vi så definere en funktion for simuleringen.
N=500 #Amount of people to simulate.
def sim(SU,B,beta):
    
    pk, V_work, V_school, ev0=sol_bel(SU,B,beta)
    
    idx = np.tile(np.arange(1,N+1),(T,1))  # Index
    t = np.tile(np.arange(1,T+1),(N,1)).T # Tidsperiode for hvert individ.

    x=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
    x1=np.zeros([T,N]) # states for all the simulated people in all the simulated time. 
    d=np.nan + np.zeros((T,N)) # decisions for all people. 1 = work, 0 = school. 
    u_init=0
#u_s=0
#u_w=0
    u_s=np.random.rand(T,N) # Fejlled for schooling
    u_w=np.random.rand(T,N) # Fejlled for work

#pks=np.zeros([max_HK,N,T])
#print(pks)
    x[0,:]=u_init # First state variable. 



    #+V_school[int(x[it,n]),it]
    for n in range(N):
        for it in range(T):
            d[it,n] = u_w[it,n]+V_work[int(x[it,n]),it]>V_school[int(x[it,n]),it]+u_s[it,n] # d=1=work. jo mindre u bliver jo mere ssh for work. Så vel i virkeligheden 
            x1[it,n] = np.minimum(x[it,n]+(1-d[it,n]),max_HK-1)
            if it < T-1:
                x[it+1,n] = x1[it,n] 

    idx =  np.reshape(idx,T*N,order='F')
    t = np.reshape(t,T*N,order='F')
    d = np.reshape(d,T*N,order='F')
    x = np.reshape(x,T*N,order='F')
    x1 = np.reshape(x1,T*N,order='F')
#dx1 = np.reshape(dx1,T*N,order='F')


    data = {'id': idx,'t': t, 'd': d, 'x': x, 'x1': x1}
    df= pd.DataFrame(data)

    pivot = pd.pivot_table(data=df, index=['t'], values=['d'], aggfunc='count')
    pivot

    df1=df.groupby('t', as_index=False).agg({"d": "sum"})
    
    return df1


In [169]:
h=sim(SU,B,beta)
print(h)

   t      d
0  1    0.0
1  2   62.0
2  3  296.0


In [86]:
j=h.iloc[2,1]
print(j)

296.0


In [170]:
def sim_mom(SU,B,beta):
    
    df1=sim(SU,B,beta)
    
    sim_mean1=df1.iloc[0,1]/N
    sim_mean2=df1.iloc[1,1]/N
    sim_mean3=df1.iloc[2,1]/N
    sim_mom1=float(sim_mean1)
    sim_mom2=float(sim_mean2)
    sim_mom3=float(sim_mean3)
    return sim_mom1, sim_mom2, sim_mom3



In [171]:
#Data moment
print(sim_mom(SU,B,beta))
dat_mom1=0.1
dat_mom2=0.2
dat_mom3=0.6
W_hat1_1 = np.eye(3)
print(W_hat1_1)

(0.0, 0.124, 0.592)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [172]:
#Define error vector, which takes the difference between simulated moment and data moment.

def err_vec(SU,B,beta):
    
    dat_mom=np.array([[dat_mom1], [dat_mom2], [dat_mom3]])
    sim_m1,sim_m2,sim_m3=sim_mom(SU,B,beta)
    sim_m=np.array([[sim_m1], [sim_m2], [sim_m3]])
    err_vec=(sim_m-dat_mom)/dat_mom
    return err_vec

In [173]:
er=err_vec(SU,B,beta)
print(er)

[[-1.        ]
 [-0.38      ]
 [-0.01333333]]


In [187]:
def criterion(params):
    #SU=design_variables[0]
    #B=design_variables[1]
    #beta=design_variables[2]
    #h=5*SU**2+4*B**2+4*beta**2
    SU, B, beta = params
    err=err_vec(SU,B,beta)
    crit_val = np.dot(np.dot(err.T, W_hat1_1), err)
    #return h
    return crit_val



In [186]:
#print(criterion(params))

NameError: name 'params' is not defined

In [4]:
import scipy.optimize as opt

In [54]:
#Prøv at bygge videre på denne.
def crit(design_variables):
    a = design_variables[0]
    s = design_variables[1]
    d = design_variables[2]
    pk, V_work, V_school, ev0=sol_bel(a,s,d)
    df=sim(a,s,d)
    Nv=df.iloc[2,1]
    #cv=5*a**2+4*s**2+4*d**2
    #return cv
    return V_work[0,0]
    #return Nv 
    

In [55]:
a_init=3
s_init=5
d_init=90
par_init= np.array([s_init, a_init, d_init])
resu=opt.minimize(crit, par_init, args=(),
                          method='L-BFGS-B',
                          bounds=((1e-10, None), (1e-10, None), (1e-10, None)))
a_SMM1_1, s_SMM1_1, d_SMM1_1 = resu.x
print(a_SMM1_1,s_SMM1_1,d_SMM1_1)

pkt, V_workt, V_schoolt, ev0t=sol_bel(a_SMM1_1,s_SMM1_1,d_SMM1_1)



print(V_workt[0,0])

2.598450590320056 1.5922008895012025 1e-10
-0.008145461715831123


In [188]:
SU_initi=0.7
B_initi=0.5
beta_initi=1.0
params_init_1 = np.array([SU_initi, B_initi, beta_initi])
smm_args1_1 = (T,N,max_HK)
results1_1 = opt.minimize(criterion, params_init_1, args=(),
                          method='L-BFGS-B',
                          bounds=((1e-10, None), (1e-10, None), (1e-10, None)))
SU_SMM1_1, B_SMM1_1, beta_SMM_1 = results1_1.x
print(SU_SMM1_1, B_SMM1_1, beta_SMM_1)

ValueError: `f0` passed has more than 1 dimension.

In [120]:
#Det virker sgu...

SU_init=0.9
result=opt.minimize_scalar(criterion, bracket=None, bounds=None, args=(B,beta,T,max_HK,num_points,dat_mom,N), method='brent', tol=None, options=None)
SU_SMM = result.x
print(SU_SMM)

print(criterion(SU_SMM,B,beta,T,max_HK,num_points,dat_mom,N))


TypeError: criterion() takes 6 positional arguments but 8 were given

In [48]:
mu_init_1 = 300
sig_init_1 = 30
params_init_1 = np.array([mu_init_1, sig_init_1])
W_hat1_1 = np.eye(2)
smm_args1_1 = (pts, unif_vals_2, cut_lb, cut_ub, W_hat1_1)
results1_1 = opt.minimize(criterion, params_init_1, args=(smm_args1_1),
                          method='L-BFGS-B',
                          bounds=((1e-10, None), (1e-10, None)))
mu_SMM1_1, sig_SMM1_1 = results1_1.x
print('mu_SMM1_1=', mu_SMM1_1, ' sig_SMM1_1=', sig_SMM1_1)

NameError: name 'pts' is not defined